In [5]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from collections import defaultdict
from scipy import stats
import matplotlib.pyplot as plt
from collections import Counter
import networkx as nx
from multiomics_benchmark import helper_functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Define your datasets
datasets = ['brain','breast', 'colorectal', 'glioma', 'head_neck', 'kidney_renal','lung', 
            'lung_squamous',  'omics', 'ovarian', 'pan_kidney', 'PreE', 
            'stomach_esophageal', 'thyroid', 'uterine']

# Define omics names for each dataset
options = defaultdict(lambda: ["Methylation", "miRNA", "RNA", "RPPA", "SCNV"], 
                      {'omics': ['cfRNA', 'proteo_lum', 'serum_lum', 'microbiome', 'CyTOF', 'metabolome', 'proteo_som'],
                       'PreE': ['cfRNA', 'lipidome', 'metabol-plasma', 'metabol-urine', 'proteome', 'microbiome'],
                       'glioma': ["Methylation", "miRNA", "RNA", "RPPA", "SCNV"],
                       'brain': ["Methylation", "miRNA", "RNA", "RPPA", "SCNV"],
                       'ovarian': ["Methylation", "miRNA", "RNA", "RPPA", "SCNV"],
                       'uterine':["Methylation", "miRNA", "RPPA", "SCNV"]
                      })

# Calculate accuracy

In [7]:
#What you want your accuracy threshold to be
r_limit = 0.5
r_limit_name = 'limit_{}'.format(r_limit)

r_p_all = []
n_samples = 50

In [8]:
for dataset_index in tqdm(range(len(datasets))):
    
    dataset = datasets[dataset_index]
    omics = options[dataset]

    os.chdir('/home/mxenoc/workspace/multiomics-benchmark/results/'+dataset+'/')

    feats_list = []

    for i in range(len(omics)): feats_list.append(np.tile(omics[i],n_samples))
        
    feats = np.hstack(feats_list)

    EN = dict()
    GFA = dict()
    Lasso = dict()
    MT_Lasso = dict()
    RF = dict()
    group_lasso = dict()
    blockForest = dict()
    ridge = dict()

    mdl =      [EN,    Lasso,   group_lasso,   RF,    ridge,   blockForest,   GFA,   MT_Lasso]
    mdl_name = ['EN', 'Lasso', 'group_lasso', 'RF',  'ridge', 'blockForest', 'GFA', 'MT_Lasso']

    for k in range(len(mdl_name)):
        with open(f"{mdl_name[k]}.pkl", 'rb') as f:
            mdl[k] = pickle.load(f)

    r_p_list = []
    models_res = []
    res = []    

    for i in range(len(mdl_name)-2): #loop through the models

        all_features = defaultdict(list)

        for l in range(len(omics)):   #loop through the modalities
            each_feature = defaultdict(list)

            for k in range(n_samples):   #loop through the targets

                x = mdl[i]['observed_test'][l][k]
                y = mdl[i]['prediction_test'][l][k]
                y.iloc[0,0] += 0.0000000001

                each_feature['r'].append(stats.spearmanr(x, y)[0])                
                each_feature['p'].append(stats.spearmanr(x, y)[1])

            all_features['r'].append(each_feature['r'])
            all_features['p'].append(each_feature['p'])

        r = np.hstack(all_features['r'])
        p = np.hstack(all_features['p'])

        r_p = pd.DataFrame({'r': r, 'p': p, 'omics': feats, 'model': mdl_name[i], 'data' : dataset})
        r_p = r_p[(r_p['r']>r_limit) & (r_p['p']<=0.05)]
        r_p_list.append(r_p)
        res.append(len(r_p))

    for n_GFA in ([len(mdl_name)-2, len(mdl_name)-1]):

        all_features = defaultdict(list)

        for l in range(len(omics)):
            each_feature = defaultdict(list)

            for k in range(n_samples):

                x = mdl[n_GFA]['observed_test'][l][k]
                y = mdl[n_GFA]['prediction_test'][l].iloc[:,k]
                                
                each_feature['r'].append(stats.spearmanr(x, y)[0])                
                each_feature['p'].append(stats.spearmanr(x, y)[1])

            all_features['r'].append(each_feature['r'])
            all_features['p'].append(each_feature['p'])

        r = np.hstack(all_features['r'])
        p = np.hstack(all_features['p'])

        r_p = pd.DataFrame({'r': r, 'p': p, 'omics': feats, 'model': mdl_name[n_GFA], 'data' : dataset})
        r_p = r_p[(r_p['r']>r_limit) & (r_p['p']<=0.05)]
        r_p_list.append(r_p)
        res.append(len(r_p))

    results = pd.DataFrame({'features': np.hstack(res), 'models': np.hstack(mdl_name), 'dataset': dataset})
    results['percentage'] = results['features']/(len(omics)*n_samples)*100

    r_p_all.append(pd.concat(r_p_list))
    
    os.chdir('/home/mxenoc/workspace/multiomics-benchmark/plots/accuracy_calculations/'+r_limit_name)
    with open(dataset+'.pkl', 'wb') as f:  
        pickle.dump(results, f)


100%|██████████| 15/15 [03:46<00:00, 15.13s/it]


# Calculate accuracy for individual models

In [10]:
r_p_all = []
individual_totals = {}

r_limit = 0.5
r_limit_name = 'limit_{}'.format(r_limit)

#for dataset in datasets:
for dataset in datasets:

    omics = options[dataset]

    with open('/home/mxenoc/workspace/multiomics-benchmark/results/'+dataset+'/individual/Lasso.pkl', 'rb') as f:
        mdl = pickle.load(f)

    r_p_list = []
    models_res = []
    res = []    

    for omic_dataset in range(len(omics)):
        all_features = defaultdict(list)
        for k in range(n_samples):
            each_feature = defaultdict(list)
            for l in range(len(omics)-1):                

                x = mdl['observed_test'][omic_dataset][k]
                y = mdl['prediction_test'][omic_dataset][k][l]

                each_feature['r'].append(stats.spearmanr(x,y)[0])                
                each_feature['p'].append(stats.spearmanr(x,y)[1])
                each_feature['predictor'].append(omics[l])

            all_features['r'].append(each_feature['r'])
            all_features['p'].append(each_feature['p'])
            all_features['predictor'].append(each_feature['predictor'])

        r = np.hstack(all_features['r'])
        p = np.hstack(all_features['p'])
        predictors = np.hstack(all_features['predictor'])

        r_p = pd.DataFrame({'r': r, 'p': p, 'predictor': predictors, 'target': omics[omic_dataset]})
        r_p = r_p[(r_p['r']>r_limit) & (r_p['p']<=0.05)]
        r_p_list.append(r_p)
        res.append(len(r_p))

    test = pd.concat(r_p_list)
    test = test.drop(['r', 'p'], axis = 1)

    dataset_predictability = Counter(test['target'])
    dataset_predictability = pd.DataFrame(dataset_predictability.items())
    dataset_predictability.columns = ['features', 'group']

    dataset_predictors = Counter(test['predictor'])
    dataset_predictors = pd.DataFrame(dataset_predictors.items())
    dataset_predictors.columns = ['features', 'group']

    links = pd.DataFrame(test.groupby(test.columns.tolist(),as_index=False).size())
    links.reset_index(level=0, inplace=True)
    links.reset_index(level=0, inplace=True)
    columns_titles = ["predictor","target", 0]
    links = links.reindex(columns = columns_titles)
    links.columns = ['var1', 'var2', 'weight']

    ###DO NOT USE THIS FOR MULTIOMICS AND PREE
    links['var1'] = links['var1'].str.partition('_')[0]
    links['var2'] = links['var2'].str.partition('_')[0]
    #######

    G = nx.from_pandas_edgelist(links, 'var1', 'var2', edge_attr=True, create_using=nx.DiGraph())

    # Edges you want to include
    threshold = 0

    # Edges you want to plot
    edges_filtered = [(u,v) for (u, v, e) in G.edges(data=True) if e['weight'] > threshold]
    weights_filtered = [e['weight'] for (u, v, e) in G.edges(data=True) if e['weight'] > threshold]

    # Choose the layout
    pos = nx.spring_layout(G, scale = 0.5, k = 0.65)

    node_colors = dataset_predictability.set_index('features')
    # Reindex your nodes to match the graph's nodes
    node_colors_reind = node_colors.reindex(G.nodes())
    #node_colors_reind['group'] = pd.Categorical(node_colors_reind['group'])
    node_colors_percent = (node_colors_reind/((len(omics)-1)*50))*100

    popular_predictors = dataset_predictors.set_index('features')
    # Reindex your nodes to match the graph's nodes
    popular_predictors_reind = popular_predictors.reindex(G.nodes())
    #node_colors_reind['group'] = pd.Categorical(node_colors_reind['group'])
    popular_predictors_percent = (popular_predictors_reind/((len(omics)-1)*50))*100

    individual_totals[dataset] = popular_predictors_percent

In [11]:
with open('/home/mxenoc/workspace/multiomics-benchmark/results/individual_totals.pkl', 'wb') as f:  
    pickle.dump(individual_totals, f)

# Calculate accuracy for stacked models

In [12]:
# Create an empty list for the results
results_all = list()
n_samples = 50

In [13]:
metalearner_list = list()
metalearner_list.append(['opt', 'Lasso', False])
metalearner_list.append(['opt', 'Lasso', True])
metalearner_list.append(['opt', 'EN', False])
metalearner_list.append(['opt', 'EN', True])
metalearner_list.append(['regular', 'RF', False])
metalearner_list.append(['opt', 'Ridge', False])
metalearner_list.append(['nnls', 'nnls', False])

In [ ]:
metalearner_list = list()
metalearner_list.append(['regular', 'lasso', False])
metalearner_list.append(['regular', 'lasso', True])
metalearner_list.append(['regular', 'EN', False])
metalearner_list.append(['regular', 'EN', True])
metalearner_list.append(['regular', 'RF', False])
metalearner_list.append(['regular', 'ridge', False])
metalearner_list.append(['nnls', 'nnls', False])

In [14]:
r_p_all = []

for dataset_index in tqdm(range(len(datasets))):

    dataset = datasets[dataset_index]
    omics = options[dataset]

    os.chdir('/home/mxenoc/workspace/multiomics-benchmark/results/'+dataset)

    feats_list = []

    for i in range(len(omics)): 
        feats_list.append(np.tile(omics[i],n_samples))

    feats = np.hstack(feats_list)

    metalearner_names= list()
    for k in range(len(metalearner_list)):
        metalearner_names.append(metalearner_list[k][0]+'_'+metalearner_list[k][1]+'_'+str(metalearner_list[k][2]))

    with open("Stacked_Lasso.pkl", 'rb') as f:
        mdl = pickle.load(f)

    r_p_list = []
    #res = defaultdict(list)
    models_res = []

    res = []    
    for metalearners in range(len(metalearner_names)):
        all_features = defaultdict(list)
        for l in range(len(omics)):
            each_feature = defaultdict(list)

            for k in range(n_samples):

                x = mdl['observed_test'][l][k]
                y = pd.concat(mdl['prediction_test'][l][k][metalearner_names[metalearners]])
                y.iloc[0,0] += 0.0000000001

                each_feature['r'].append(stats.spearmanr(x,y)[0])                
                each_feature['p'].append(stats.spearmanr(x,y)[1])

            all_features['r'].append(each_feature['r'])
            all_features['p'].append(each_feature['p'])

        r = np.hstack(all_features['r'])
        p = np.hstack(all_features['p'])

        r_p = pd.DataFrame({'r': r, 'p': p, 'omics': feats, 'model': metalearner_names[metalearners], 'data' : dataset})
        r_p = r_p[(r_p['r']>r_limit) & (r_p['p']<=0.05)]
        r_p_list.append(r_p)
        res.append(len(r_p))

    results = pd.DataFrame({'features': np.hstack(res), 'models': np.hstack(metalearner_names), 'dataset': dataset})
    results['percentage'] = results['features']/(len(omics)*n_samples)*100

    results_all.append(results)
    
    r_p_all.append(pd.concat(r_p_list))

    os.chdir('/home/mxenoc/workspace/multiomics-benchmark/plots/accuracy_calculations/'+r_limit_name+'/stacked')
    with open(dataset+'.pkl', 'wb') as f:  
        pickle.dump(results, f)


100%|██████████| 15/15 [06:47<00:00, 27.20s/it]
